<a href="https://colab.research.google.com/github/shivaprajapati34390-netizen/ML-project/blob/main/stock_price_prediction_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date,timedelta
today=date.today()

In [24]:
d1=today.strftime("%Y-%m-%d")
end_date=d1
d2=date.today()-timedelta(days=5000)
d2=d2.strftime("%Y-%m-%d")

data=yf.download('AAPL',start=d2,end=end_date,progress=False, auto_adjust=True)

# Explicitly handle potential MultiIndex after yf.download
# yfinance can return a MultiIndex where the first level is the metric (e.g., 'Open')
# and the second level is the ticker (e.g., 'AAPL'). We want the metric.
if isinstance(data.columns, pd.MultiIndex):
    data.columns = [col[0] for col in data.columns]

# Standardize column names to lowercase and replace spaces/dots
data.columns = [col.lower().replace(' ', '_').replace('.', '') for col in data.columns]

# Ensure 'close' column exists; if 'adj_close' is present and 'close' is not, rename it
if 'adj_close' in data.columns and 'close' not in data.columns:
    data.rename(columns={'adj_close': 'close'}, inplace=True)

data["date"] = data.index.map(lambda x: x.strftime('%Y-%m-%d')) # Convert datetime index to string 'YYYY-MM-DD'

# Define the columns we want to keep
required_cols = ["date","open","high","low","close","volume"]

# Filter the DataFrame to only include the required columns that actually exist
data = data[[col for col in required_cols if col in data.columns]]

data.reset_index(drop=True,inplace=True)
print(data.tail())

            date        open        high         low       close    volume
3436  2026-02-03  268.948351  271.625839  267.359811  269.228088  64394700
3437  2026-02-04  272.035451  278.689229  272.035451  276.231506  90545700
3438  2026-02-05  277.869995  279.238709  272.974582  275.652069  52977400
3439  2026-02-06  276.860920  280.647386  276.671095  277.859985  50453400
3440  2026-02-09  277.910004  278.200012  271.700012  274.619995  44562300


In [7]:
# Candlestick chart gives the clear picture of incresea and decrease in stock price so lets visulaize the candlestick chart of the data before moving further

In [27]:
from plotly.graph_objs import XAxis
import plotly.graph_objects as go
figure=go.Figure(data=[go.Candlestick(x=data["date"], open=data["open"], high=data["high"], low=data["low"], close=data["close"])])
figure.update_layout(title="Apple stock price analysis",xaxis_rangeslider_visible=False)
figure.show()

In [25]:
import plotly.express as px

# Calculate the correlation coefficient using flattened column names
correlation = data['close'].corr(data['volume'])
print(f"The correlation between Close Price and Volume is: {correlation:.4f}")

# Create a scatter plot to visualize the relationship using flattened column names
figure = px.scatter(data, x="volume", y="close", title="Close Price vs. Volume")
figure.update_layout(xaxis_title="Volume", yaxis_title="Close Price (USD)")
figure.show()

The correlation between Close Price and Volume is: -0.5451


### Correlation Analysis

The correlation coefficient ranges from -1 to 1:

*   **1**: Indicates a perfect positive linear relationship (as one variable increases, the other increases proportionally).
*   **-1**: Indicates a perfect negative linear relationship (as one variable increases, the other decreases proportionally).
*   **0**: Indicates no linear relationship.

A positive correlation suggests that as trading volume increases, the closing price tends to increase, and vice-versa. A negative correlation would suggest the opposite. A correlation close to zero means there's little to no linear relationship between these two factors. The scatter plot provides a visual confirmation of this relationship.

In [30]:
import plotly.graph_objects as go

# Create a line plot for the closing price
figure = go.Figure()
figure.add_trace(go.Scatter(x=data["date"], y=data["close"], mode='lines', name='Close Price'))

# Calculate a 200-day Simple Moving Average (SMA) for long-term trend
data['SMA_200'] = data['close'].rolling(window=200).mean()
figure.add_trace(go.Scatter(x=data["date"], y=data["SMA_200"], mode='lines', name='200-Day SMA', line=dict(color='orange', width=2)))

figure.update_layout(title="Apple Stock Price Trend Analysis with 200-Day SMA",
                     xaxis_title="Date",
                     yaxis_title="Close Price (USD)",
                     xaxis_rangeslider_visible=False)
figure.show()

### Overall Trend Analysis

Looking at the generated chart, especially with the 200-day Simple Moving Average (SMA), we can observe the following about Apple's stock price trend:

*   **Long-term Growth**: Over the several years represented in the `data` DataFrame, Apple's stock price shows a significant upward trend, indicating long-term growth. The 200-day SMA, which smooths out daily fluctuations, consistently moves upwards, confirming this long-term bullish sentiment.

*   **Periods of Volatility and Corrections**: While the overall trend is upward, there are noticeable periods where the stock price deviates from the SMA. These could represent market corrections, economic downturns, or specific company news that temporarily impact the stock. However, in most cases, the price tends to revert back towards and continue above the 200-day SMA, reinforcing the uptrend.

*   **SMA as a Trend Indicator**: The 200-day SMA acts as a key indicator of the long-term trend. When the stock price is consistently above the 200-day SMA, it typically signifies an uptrend. Conversely, if the price drops below the 200-day SMA for an extended period, it could signal a reversal or a downtrend.

In summary, based on the `data` DataFrame and the visual analysis, Apple's stock has exhibited a strong, sustained upward trend over the observed period, despite intermittent periods of volatility.

In [32]:
correlation = data.drop('date', axis=1).corr()
print(correlation["close"].sort_values(ascending=False))

close      1.000000
high       0.999890
low        0.999885
open       0.999756
SMA_200    0.985384
volume    -0.545129
Name: close, dtype: float64


In [33]:
# Training LSTM(long short term memory) for stock price prediction
# now i will start with  training LSTM   model for stock price lets first split the data into traning sets and test sets

In [35]:
x=data[["open","high","low","volume"]]
y=data["close"]
x=x.to_numpy()
y=y.to_numpy()
y=y.reshape(-1,1)

In [47]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)

# Reshape xtrain and xtest for LSTM input: (num_samples, timesteps, features)
# Here, timesteps = num_features (4), and features_per_timestep = 1
xtrain = xtrain.reshape(xtrain.shape[0], xtrain.shape[1], 1)
xtest = xtest.reshape(xtest.shape[0], xtest.shape[1], 1)

In [38]:
#  now i will prepare the neural network architecture for LSTM:

In [40]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM
model=Sequential()
model.add(LSTM(128,return_sequences=True,input_shape=(xtrain.shape[1],1)))
model.add(LSTM(64,return_sequences=False))
model.add(Dense (25))
model.add(Dense(1))
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 4, 128)         │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,625 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 117,619 (459.45 KB)

 Trainable params: 117,619 (459.45 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
#  now here is how we can train our neural network model for stock price prediction

In [42]:
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(xtrain,ytrain,batch_size=1,epochs=30)

Epoch 1/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 3084.7117
Epoch 2/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 34.0311
Epoch 3/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 30.5836
Epoch 4/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 19.8282
Epoch 5/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 20.6083
Epoch 6/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 23.0909
Epoch 7/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 18.6105
Epoch 8/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 25.0735
Epoch 9/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 13.1845
Epoch 10/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 14.2016
Epoch 11/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 25.4332
Epoch 12/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 20.6082
Epoch 13/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 17.3999
Epoch 14/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 20.1053

In [43]:
#  now lets check the model by giving inputs values according to the features that we have used to train this model and predicting the final result

In [48]:
import numpy as np
#features = [Open, High, Low, Adj Close, Volume]
features = np.array([[177.32165415, 180.419998, 177.070007, 74919600]])
model.predict(features)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


array([[176.79254]], dtype=float32)

In [ ]:
# So, this 176.79254 is the model's best estimate for the closing price, given the specific market conditions (open, high, low, and volume) you fed into it.
